In [302]:
import pandas as pd
import numpy as np
import os

In [303]:
DATA_DIR = "data/"
FILE_NAME = "data.csv"
FINAL_DATA = "rearranged_data.csv"

In [304]:
# Load data
df = pd.read_csv(os.path.join(DATA_DIR, FILE_NAME), delimiter=";")

In [305]:
# function to copy serial
def copy_serial(row):
    if not pd.isnull(row["ZG04"]):
        row["SERIAL"] = row["ZG04"]
    elif not pd.isnull(row["ZG05"]):
        row["SERIAL"] = row["ZG05"]
    return row

In [306]:
# move serial to serial from w01
df = df.apply(lambda row: copy_serial(row), axis=1)

In [307]:
# Drop lines where we have no serial number
df = df[~pd.isnull(df["SERIAL"])]

In [308]:
# Function to extract group
serial_group = dict()
def extract_variable(row):
    if not pd.isnull(row["ZG04"]):
        serial_group.update({row["SERIAL"]:"MS"})
    elif not pd.isnull(row["ZG05"]):
        serial_group.update({row["SERIAL"]:"AL"})
    
def assign_variable(var):
    return serial_group[var]

In [309]:
# Extract group
df.apply(lambda row: group_variable(row), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
3747    None
3748    None
3749    None
3750    None
3751    None
Length: 3717, dtype: object

In [310]:
# Drop some unnecessary columns
df.drop(["MODE", "REF", "CASE", "FINISHED", "Q_VIEWER", "MAILSENT", "LASTPAGE", "MAXPAGE", "MISSING", "MISSREL"], axis=1, inplace=True)

In [311]:
# Find all cases that have completed all the stuff
def collect_complete_cases():
    complete_cases = []
    for indiv in df["SERIAL"].unique():
        if df[df["SERIAL"]== indiv].shape[0] == 12:
            complete_cases.append(indiv)
        
df = df[df["SERIAL"].isin(complete_cases)]

In [312]:
value_vars_drop = ['SERIAL',"QUESTNNR",'STARTED','TIME001','TIME002','TIME003','TIME004','TIME005','TIME006','TIME007','TIME008',
 'TIME009','TIME010','TIME011','TIME012','TIME013','TIME014','TIME015','TIME016','TIME017','TIME018',
 'TIME019','TIME020','TIME021','TIME022','TIME023','TIME024','TIME025','TIME026','TIME027','TIME028','TIME029',
 'TIME030','TIME031','TIME032','TIME033','TIME034','TIME035','TIME036','TIME_SUM','LASTDATA','TIME_RSI','DEG_TIME', "ZG04", "ZG05"]
value_vars = [x for x in df.columns if x not in value_vars_drop]
df = pd.melt(df, id_vars=["SERIAL","QUESTNNR"], value_vars=value_vars)

In [313]:
# Drop variables without anser
df = df[~pd.isnull(df["value"])]

In [314]:
# Add group variable
df["GROUP"] = df["SERIAL"].apply(lambda val: serial_group[val])

In [315]:
# Rearrange table
df = df[["SERIAL", "GROUP", "QUESTNNR", "variable", "value"]]

In [320]:
# Store to new excel
df.to_excel(os.path.join(DATA_DIR, FINAL_DATA), index=False)